In [ ]:
!pip install paho-mqtt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.4/99.4 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62120 sha256=0c55a663155e3e4061a7c3625579e812c06f68141d0c78bedcd3596293e702e6
  Stored in directory: /root/.cache/pip/wheels/8b/bb/0c/79444d1dee20324d442856979b5b519b48828b0bd3d05df84a
Successfully built paho-mqtt


In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
df = pd.read_csv('data_iot.csv')
data = df.copy()

Saving data_iot.csv to data_iot.csv


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9480 entries, 0 to 9479
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         9480 non-null   int64  
 1   Month        9480 non-null   int64  
 2   Day          9480 non-null   int64  
 3   Hour         9480 non-null   int64  
 4   Temperature  9480 non-null   float64
 5   Humidity     9480 non-null   float64
 6   PRECTOTCORR  9480 non-null   float64
dtypes: float64(3), int64(4)
memory usage: 518.6 KB


In [ ]:
df.head()

,Year,Month,Day,Hour,Temperature,Humidity,PRECTOTCORR
0,2022,11,1,7,19.08,79.75,0.0
1,2022,11,1,8,21.46,71.12,0.0
2,2022,11,1,9,23.37,61.75,0.0
3,2022,11,1,10,24.33,58.25,0.0
4,2022,11,1,11,24.94,57.06,0.0


In [ ]:
df.isnull().any()

Year           False
Month          False
Day            False
Hour           False
Temperature    False
Humidity       False
PRECTOTCORR    False
dtype: bool

In [ ]:
# Chuyển đổi giá trị của cột "PRECTOTCORR"
data['PRECTOTCORR'] = data['PRECTOTCORR'].apply(lambda x: 'Yes' if x > 0 else 'No')
print(data)

      Year  Month  Day  Hour  Temperature  Humidity PRECTOTCORR
0     2022     11    1     7        19.08     79.75          No
1     2022     11    1     8        21.46     71.12          No
2     2022     11    1     9        23.37     61.75          No
3     2022     11    1    10        24.33     58.25          No
4     2022     11    1    11        24.94     57.06          No
...    ...    ...  ...   ...          ...       ...         ...
9475  2023     12    1     2        17.43     93.38         Yes
9476  2023     12    1     3        16.76     93.00         Yes
9477  2023     12    1     4        15.91     90.31         Yes
9478  2023     12    1     5        15.41     89.56         Yes
9479  2023     12    1     6        15.03     88.44         Yes

[9480 rows x 7 columns]


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [ ]:
# Xóa cột "datetime", "time", "id"
data = data.drop('Year', axis=1)
data = data.drop('Month', axis=1)
data = data.drop('Day', axis=1)
data = data.drop('Hour', axis=1)

# Chia thành các đặc trưng (features) và nhãn (label)
X = data.drop('PRECTOTCORR', axis=1)  # Đặc trưng
y = data['PRECTOTCORR']  # Nhãn

# Chuyển đổi nhãn sang dạng số nguyên
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Xây dựng mô hình SVM
model = SVC()


# # Xây dựng mô hình Linear Regression
# model = LinearRegression()

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình bằng độ chính xác
accuracy = accuracy_score(y_test, y_pred)


# # Đánh giá mô hình bằng độ chính xác
# accuracy = mean_absolute_error(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.7241561181434599


In [ ]:
import paho.mqtt.client as mqtt

In [ ]:
humi = 0
temp = 0
client = mqtt.Client()
def on_message(client, userdata, message):
    if message.topic == "ESP32WI/DHT11/Temperature01":
        message = str(message.payload.decode('utf-8'))
        temp = float(message.split('||')[0])
        humi = float(message.split('||')[1])
        print(f"Nhiệt độ: {temp}")
        print(f"Độ ẩm: {humi}")
        data = pd.DataFrame({
            'Year': 0,
            'Month': 0,
            'Day': 0,
            'Hour': 0,
            'Temperature': [temp],
            'Humidity': [humi],
        })
        # Xóa cột "datetime", "time", "id"
        data = data.drop('Year', axis=1)
        data = data.drop('Month', axis=1)
        data = data.drop('Day', axis=1)
        data = data.drop('Hour', axis=1)
        prediction = model.predict(data)
        predicted_label = label_encoder.inverse_transform(prediction)
        topic = "predict/label"
        message = str(predicted_label[0])
        client.publish(topic, message)

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Kết nối thành công")
        client.subscribe("ESP32WI/DHT11/Humidity01")
        client.subscribe("ESP32WI/DHT11/Temperature01")
    else:
        print("Kết nối thất bại, mã lỗi:", rc)

client.on_connect = on_connect
client.on_message = on_message

broker_address = "broker.mqttdashboard.com"
username = "lamlam"
password = "Aa123456"
client.username_pw_set(username, password)
client.connect(broker_address, 1883)

client.loop_forever()